In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
# from pyspark import SparkContext, SparkConf

# conf = SparkConf()
# conf.set("spark.app.name", "natasha pritykovskaya Spark Dataframe app") 
# sc = SparkContext(conf=conf)

In [3]:
# from pyspark import SQLContext

# sqlContext = SQLContext(sc)

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "ao Spark Dataframe app") 

spark = SparkSession.builder.config(conf=conf).appName("ao Spark Dataframe app").getOrCreate()

In [5]:
spark

Spark Session:

    - Spark Context
    - SQL Context
    - Hive Context
    - Streaming Context

In [6]:
sc = spark.sparkContext

In [7]:
sc

<SparkContext master=yarn appName=ao Spark Dataframe app>

In [8]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# DATA

## 1

In [9]:
dz = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]
print(dz)
id_films = [i[0] for i in dz]
print(id_films)

[[23126, 'en', 'Compass - powerful SASS library that makes your life easier'], [21617, 'en', 'Preparing for the AP* Computer Science A Exam — Part 2'], [16627, 'es', 'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, 'es', 'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, 'ru', 'Программирование на Lazarus'], [13702, 'ru', 'Математическая экономика']]
[23126, 21617, 16627, 11556, 16704, 13702]


## 2

In [10]:
! hdfs dfs -ls /labs/slaba02/DO_record_per_line.json

-rw-r--r--   3 hdfs hdfs   69519728 2021-02-27 21:58 /labs/slaba02/DO_record_per_line.json


In [11]:
! hdfs dfs -ls /labs/slaba02/

Found 1 items
-rw-r--r--   3 hdfs hdfs   69519728 2021-02-27 21:58 /labs/slaba02/DO_record_per_line.json


In [12]:
! hdfs dfs -head /labs/slaba02/DO_record_per_line.json

{"lang": "en", "name": "Accounting Cycle: The Foundation of Business Measurement and Reporting", "cat": "3/business_management|6/economics_finance", "provider": "Canvas Network", "id": 4, "desc": "This course introduces the basic financial statements used by most businesses, as well as the essential tools used to prepare them. This course will serve as a resource to help business students succeed in their upcoming university-level accounting classes, and as a refresher for upper division accounting students who are struggling to recall elementary concepts essential to more advanced accounting topics. Business owners will also benefit from this class by gaining essential skills necessary to organize and manage information pertinent to operating their business. At the conclusion of the class, students will understand the balance sheet, income statement, and cash flow statement. They will be able to differentiate between cash basis and accrual basis techniques, and know when each is appro

In [13]:
df = spark.read.json("/labs/slaba02/DO_record_per_line.json")  # .csv("/lectures/lecture02/data/ml-100k/u.user", schema=schema, sep="|")

In [14]:
df

DataFrame[cat: string, desc: string, id: bigint, lang: string, name: string, provider: string]

In [15]:
df.show()

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|
|9/humanities|15/m...|This game-based c...|  9|  en|College Foundatio...|Canvas Network|
|  14/social_sciences|What’s in your di...| 10|  en|Digital Literacies I|Canvas Network|
|  14/social_sciences|The goal of the D...| 11|  en|Digital Literacie...|Canvas Network|
|  14/social_sciences

In [16]:
df.count()

28153

# TF IDF 

In [31]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [32]:
tokenizer = Tokenizer(inputCol="desc", outputCol="words")
wordsData = tokenizer.transform(df)

In [33]:
wordsData.show()

+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
|                 cat|                desc| id|lang|                name|      provider|               words|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|[this, course, in...|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|[this, online, co...|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|[this, course, is...|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|[we, live, in, a,...|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|[this, self-paced...|
|9/humanities|15/m...|This game-based c...|  9|  en|College Foundatio...|Canvas Network|[this, game-based...|
|  14/soci

In [34]:
words = wordsData.rdd.flatMap(lambda x: x[6])
num_words = words.groupBy(lambda x: x).count()

print(num_words)

566578


In [35]:
# из текста задания
num_words = 10000

In [36]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=num_words)
featurizedData = hashingTF.transform(wordsData)

In [37]:
featurizedData.show()

+--------------------+--------------------+---+----+--------------------+--------------+--------------------+--------------------+
|                 cat|                desc| id|lang|                name|      provider|               words|         rawFeatures|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+--------------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|[this, course, in...|(10000,[36,42,63,...|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|[this, online, co...|(10000,[32,222,29...|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|[this, course, is...|(10000,[30,41,246...|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|[we, live, in, a,...|(10000,[493,572,7...|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Net

In [38]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

# COSINE

In [39]:
from pyspark.sql.types import DoubleType, FloatType
from pyspark.sql.functions import udf
import pyspark.sql.functions as f

In [40]:
# @udf
# def sim_cos(v1,v2):
#     try:
#         p = 2
#         return v1.dot(v2)/v1.norm(p)*v2.norm(p) # v1.dot(v2)/v1.norm(p)*v2.norm(p) float(v1.dot(v2))/float(v1.norm(p)*v2.norm(p))
#     except:
#         return 0

In [41]:
@udf(returnType=FloatType())
def sim_cos(v1,v2):
    try:
        p = 2
        return  float( (v1.dot(v2))) / float( (v1.norm(p)*v2.norm(p)) )
    except:
        return 0

In [42]:
# id_films

# rescaledData

# vect = rescaledData.filter(rescaledData['id'] == 23126).select(rescaledData['features'].alias(str(23126))).limit(1)
# rescaledData = rescaledData.crossJoin(vect)
# #rescaledData = rescaledData.withColumn(str(23126) + '_cosine' , sim_cos(str(23126), 'features')) 

# rescaledData

# @udf(returnType=DoubleType())
# def sim_cos(v1,v2):
#     try:
#         p = 2
#         return  float( (v1.dot(v2))/(v1.norm(p)*v2.norm(p)) ) # v1.dot(v2)/v1.norm(p)*v2.norm(p) float(v1.dot(v2))/float(v1.norm(p)*v2.norm(p))
#     except:
#         return 0

# # sim_cos = udf( (v1.dot(v2))/(v1.norm(p)*v2.norm(p)) , DoubleType() )

# rescaledData = rescaledData.withColumn(str(23126) + '_cosine' , sim_cos(str(23126), 'features')) 

# rescaledData.schema

# rescaledData.show()

In [44]:
for i in id_films:
    vect = rescaledData.filter(rescaledData['id'] == i).select(rescaledData['features'].alias(str(i))).limit(1)
    rescaledData = rescaledData.crossJoin(vect)
    rescaledData = rescaledData.withColumn(str(i) + '_cosine' , sim_cos(str(i), 'features') )

In [45]:
rescaledData

DataFrame[cat: string, desc: string, id: bigint, lang: string, name: string, provider: string, words: array<string>, rawFeatures: vector, features: vector, 23126: vector, 23126_cosine: float, 21617: vector, 21617_cosine: float, 16627: vector, 16627_cosine: float, 11556: vector, 11556_cosine: float, 16704: vector, 16704_cosine: float, 13702: vector, 13702_cosine: float]

In [46]:
tmp = rescaledData.limit(20).toPandas()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [47]:
tmp.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
cat,3/business_management|6/economics_finance,11/law,5/computer_science|15/mathematics_statistics_a...,14/social_sciences,2/biology_life_sciences,9/humanities|15/mathematics_statistics_and_dat...,14/social_sciences,14/social_sciences,14/social_sciences,14/social_sciences,12/medicine_health,16/languages,4/chemistry,8/engineering_technology,1/arts_music_film,9/humanities,14/social_sciences,9/humanities,8/engineering_technology,14/social_sciences
desc,This course introduces the basic financial sta...,This online course will introduce you to Ameri...,This course is taught in French Vous voulez co...,We live in a digitally connected world. The wa...,This self-paced course is designed to show tha...,This game-based course provides prospective st...,What’s in your digital teaching toolbox? Do yo...,The goal of the Digital Literacy 2 course is t...,Ready to explore Web-based tools to ignite stu...,This self-paced course provides participants w...,What is “interprofessional collaborative pract...,This course presents the academic foundations ...,Chemistry is an integral part of our lives and...,Are you considering a career in engineering? A...,Princess stories have been popular for centuri...,This first installment of a four-part series i...,This course examines the political (world orde...,This course will explore government policies d...,The field of technical writing has come a long...,Are you a Higher Ed instructor or K-12 educato...
id,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
lang,en,en,fr,en,en,en,en,en,en,en,en,en,en,en,en,en,en,en,en,en
name,Accounting Cycle: The Foundation of Business M...,American Counter Terrorism Law,Arithmétique: en route pour la cryptographie,Becoming a Dynamic Educator,Bioethics,"College Foundations: Reading, Writing, and Math",Digital Literacies I,Digital Literacies II,Digital Tools for the K-12 Educator,Discover Your Value: Turning Experience into C...,Enhancing Patient Safety through Interprofessi...,Ethics and Values in a Multicultural World,Exploring Chemistry,Exploring Engineering,Fairy Tales: Origins and Evolution of Princess...,First Peoples to the Early Republic: Born in C...,Forums for a Future,From the Gilded Age to the Roaring Twenties: ...,Fundamentals of Structured Writing for Technic...,Hybrid Courses: Best of Both Worlds
provider,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network,Canvas Network
words,"[this, course, introduces, the, basic, financi...","[this, online, course, will, introduce, you, t...","[this, course, is, taught, in, french, vous, v...","[we, live, in, a, digitally, connected, world....","[this, self-paced, course, is, designed, to, s...","[this, game-based, course, provides, prospecti...","[what’s, in, your, digital, teaching, toolbox?...","[the, goal, of, the, digital, literacy, 2, cou...","[ready, to, explore, web-based, tools, to, ign...","[this, self-paced, course, provides, participa...","[what, is, “interprofessional, collaborative, ...","[this, course, presents, the, academic, founda...","[chemistry, is, an, integral, part, of, our, l...","[are, you, considering, a, career, in, enginee...","[princess, stories, have, been, popular, for, ...","[this, first, installment, of, a, four-part, s...","[this, course, examines, the, political, (worl...","[this, course, will, explore, government, poli...","[the, field, of, technical, writing, has, come...","[are, you, a, higher, ed, instructor, or, k-12..."
rawFeatures,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.

In [48]:
# по метрике (убывание) => по названию (лексикографически по возрастанию) => по возрастанию id.

In [49]:
qq = rescaledData.orderBy(f.col("11556_cosine").desc(),f.col("name").asc(),f.col("id").asc())

In [50]:
# qq = rescaledData.orderBy(f.col("11556_cosine").desc())

In [51]:
qq.limit(10).toPandas().T

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


,0,1,2,3,4,5,6,7,8,9
cat,,,,9/humanities,,15/mathematics_statistics_and_data_analysis,,,,
desc,La transformación del aula con el Aprendizaje...,El Aprendizaje Basado en Problemas como técni...,Mejora tu competencia docente a través de las...,Analizar diferentes casos de estudio de usos e...,Ocupa tu dispositivo a favor de tus estudios ...,El curso propone un acercamiento a la Matemáti...,Herramientas tecnológicas Este curso esta dis...,Desarrolla y perfecciona tus competencias doc...,Aprenda a la poderosa herramienta que ha revo...,Capta y profundiza el interés de tus estudian...
id,11556,16488,13461,468,10447,387,22710,9289,5936,23357
lang,es,es,es,es,es,es,es,es,es,es
name,Aprendizaje Colaborativo by UNID Universidad I...,Aprendizaje basado en problemas by UNID Univer...,TIC para la enseñanza by Julia García Escudero,Tecnologías de información y comunicación en l...,Mejores calificaciones usando tu dispositivo m...,Matemáticas y Movimiento,Aplicaciones creativas en el aula - Udemy,Habilidades educativas para la docencia del si...,HTML5 by Francisco Javier Arce Anguiano,Diseña Cursos Dinámicos que Fascinan a Tus Est...
provider,Udemy,Udemy,Udemy,Coursera,Udemy,Coursera,Udemy,Udemy,Udemy,Udemy
words,"[, la, transformación, del, aula, con, el, apr...","[, el, aprendizaje, basado, en, problemas, com...","[, mejora, tu, competencia, docente, a, través...","[analizar, diferentes, casos, de, estudio, de,...","[, ocupa, tu, dispositivo, a, favor, de, tus, ...","[el, curso, propone, un, acercamiento, a, la, ...","[, herramientas, tecnológicas, este, curso, es...","[, desarrolla, y, perfecciona, tus, competenci...","[, aprenda, a, la, poderosa, herramienta, que,...","[, capta, y, profundiza, el, interés, de, tus,..."
rawFeatures,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
features,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
23126,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [52]:
res_dict = {}
cources = []
dfs = []

In [53]:
for i in id_films:
    top = rescaledData.orderBy(f.col(str(i) + "_cosine").desc(),f.col("name").asc(),f.col("id").asc())
    top_10 = top.select(["id"]).limit(10)
    cources.append(rescaledData.filter(rescaledData["id"].isin([i])))
    dfs.append(top.limit(10).toPandas())
    top_10_list = [int(row.id) for row in top_10.collect()]
    print( top_10_list )
    res_dict[str(i)] = top_10_list
    
#     vect = rescaledData.filter(rescaledData['id'] == i).select(rescaledData['features'].alias(str(i))).limit(1)
#     rescaledData = rescaledData.crossJoin(vect)
#     rescaledData = rescaledData.withColumn(str(i) + '_cosine' , sim_cos(str(i), 'features') )

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


[23126, 13782, 13665, 24419, 20638, 2724, 25782, 2633, 2723, 13348]
[21617, 21609, 21608, 21616, 21492, 21624, 21623, 21630, 21628, 21508]
[16627, 11431, 12247, 13021, 25010, 11575, 5687, 9598, 5372, 12863]
[11556, 16488, 13461, 468, 10447, 387, 22710, 9289, 5936, 23357]
[16704, 3864, 23407, 1365, 20645, 1426, 20105, 8217, 1236, 1164]
[864, 13702, 1216, 7173, 8313, 1052, 17017, 19613, 21017, 17015]


In [54]:
res_dict

{'23126': [23126, 13782, 13665, 24419, 20638, 2724, 25782, 2633, 2723, 13348],
 '21617': [21617,
  21609,
  21608,
  21616,
  21492,
  21624,
  21623,
  21630,
  21628,
  21508],
 '16627': [16627, 11431, 12247, 13021, 25010, 11575, 5687, 9598, 5372, 12863],
 '11556': [11556, 16488, 13461, 468, 10447, 387, 22710, 9289, 5936, 23357],
 '16704': [16704, 3864, 23407, 1365, 20645, 1426, 20105, 8217, 1236, 1164],
 '13702': [864, 13702, 1216, 7173, 8313, 1052, 17017, 19613, 21017, 17015]}

In [55]:
id_films

[23126, 21617, 16627, 11556, 16704, 13702]

In [56]:
cources[0].toPandas()

/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


,cat,desc,id,lang,name,provider,words,rawFeatures,features,23126,...,21617,21617_cosine,16627,16627_cosine,11556,11556_cosine,16704,16704_cosine,13702,13702_cosine
0,,Improve your SASS skill by learning benefits ...,23126,en,Compass - powerful SASS library that makes you...,Udemy,"[, improve, your, sass, skill, by, learning, b...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.036504,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.013446,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.008747,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0


In [57]:
dfs[0].T

,0,1,2,3,4,5,6,7,8,9
cat,,5/computer_science,,,1/arts_music_film,1/arts_music_film|9/humanities,,1/arts_music_film|9/humanities,1/arts_music_film|9/humanities,
desc,Improve your SASS skill by learning benefits ...,Expand your Sass abilities with the Compass ad...,Take your Sass skills to the next level. Lear...,"Learn Empowerment, Happiness, Resiliency, and...",Learn how to create new visual narratives with...,This course covers techniques of creating narr...,Learn how to clean up your CSS and write it f...,This course explores the properties of non-seq...,This course explores the properties of non-lin...,Learn to supercharge your CSS with Sass. A ea...
id,23126,13782,13665,24419,20638,2724,25782,2633,2723,13348
lang,en,en,en,en,en,en,en,en,en,en
name,Compass - powerful SASS library that makes you...,Assembling Sass Part 2,The Next Step with Sass and Compass by Lisa Ca...,Learn the 7 Mindsets to live your Ultimate Life,Introduction to Photo Compositing,Interactive and Non-Linear Narrative: Theory a...,Write Clean CSS using SASS - Udemy,Technologies of Humanism,Interactive and Non-Linear Narrative: Theory a...,Learning Sass - CSS Just Got Exciting. by Infi...
provider,Udemy,Code School,Udemy,Udemy,Lynda,MIT OpenCourseWare,Udemy,MIT OpenCourseWare,MIT OpenCourseWare,Udemy
words,"[, improve, your, sass, skill, by, learning, b...","[expand, your, sass, abilities, with, the, com...","[, take, your, sass, skills, to, the, next, le...","[, learn, empowerment,, happiness,, resiliency...","[learn, how, to, create, new, visual, narrativ...","[this, course, covers, techniques, of, creatin...","[, learn, how, to, clean, up, your, css, and, ...","[this, course, explores, the, properties, of, ...","[this, course, explores, the, properties, of, ...","[, learn, to, supercharge, your, css, with, sa..."
rawFeatures,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
features,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 2.981414576488381, 0.0, 0.0, 0.0, 0.0, 0...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
23126,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [58]:
import json

with open("lab02.json", "w") as write_file:
    json.dump(res_dict, write_file)

In [59]:
import json

with open("../../../lab02.json", "w") as write_file:
    json.dump(res_dict, write_file)

In [ ]:
desc()

In [ ]:
rescaledData

In [ ]:
rescaledData.cache()

In [ ]:
rescaledData.rdd.getNumPartitions()

In [ ]:
# vect_1 = rescaledData.filter(rescaledData['id'] == 23126).select(rescaledData['features'].alias("ref")).limit(1)
# # rescaledData = rescaledData.crossJoin(vect_1)

In [ ]:
# type( vect_1[0].features )

In [ ]:
# rescaledData.withColumn('cosine' , sim_cos('ref', 'features') ).show()

In [ ]:
@udf
def sim_cos(v1,v2):
    try:
        p = 2
        return float(v1.dot(v2))/float(v1.norm(p)*v2.norm(p))
    except:
        return 0

result = data.alias("i").join(data.alias("j"), F.col("i.ID") < F.col("j.ID"))\
    .select(
        F.col("i.ID").alias("i"),
        F.col("j.ID").alias("j"),
        sim_cos("i.feature", "j.feature").alias("sim_cosine"))\
    .sort("i", "j")
result.show()

## Как создать DataFrame?

### Прочитать из внешнего источника

In [ ]:
spark.read

### pipeline загрузки
```python
spark.read\
     .format(...)\
     .option(key, value)\
     .option(key, value)\
     .load(path)
```

In [ ]:
df = spark.read\
          .format("csv")\
          .option("sep", "|")\
          .load("/lectures/lecture02/data/ml-100k/u.user")

In [ ]:
df

In [ ]:
df.show(1)

In [ ]:
df.take(5)

### Schema!

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [ ]:
schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("age", IntegerType()),
    StructField("gender", StringType()),
    StructField("occupation", StringType()),
    StructField("zip", IntegerType())
])

In [ ]:
df = spark.read\
          .schema(schema)\
          .format("csv")\
          .option("sep", "|")\
          .load("/lectures/lecture02/data/ml-100k/u.user")

In [ ]:
df

In [ ]:
df.printSchema()

In [ ]:
df.show(5)

In [ ]:
df.summary().show()

### Таким образом более приближенный к реальности pipeline выглядит как
```python
spark.read\
     .schema(schema)\
     .format(...)\
     .option(key, value)\
     .option(key, value)\
     .load(path)
```

### Есть так же удобные wrapper'ы

In [ ]:
df = spark.read.csv("/lectures/lecture02/data/ml-100k/u.user", schema=schema, sep="|")

In [ ]:
df

In [ ]:
df.show(5)

### Много источников данных с унифицированным API!
+ CSV
+ JSON
+ Hive
+ HBase
+ Cassandra
+ MySQL
+ PostgreSQL
+ Parquet
+ ORC
+ Kafka
+ ElasticSearch
+ Amazon S3
+ ...и еще больше через custom connectors

### Создадим DataFrame из RDD, pandas.DataFrame или из list

In [ ]:
rdd = sc.textFile("/lectures/lecture02/data/ml-100k/u.user").map(lambda x: x.split("|"))

In [ ]:
rdd.take(5)

In [ ]:
df = spark.createDataFrame(rdd)

In [ ]:
df

In [ ]:
df = spark.createDataFrame(rdd, schema=schema)

In [ ]:
df

In [ ]:
df.show(5)

### Может быть с неправильными типами можно справится путем игнорирования верификации schema???

In [ ]:
df = spark.createDataFrame(rdd, schema=schema, verifySchema=False)

In [ ]:
df.show(5)

### Не выходит требуется конвертировать в правильные типы

In [ ]:
rdd = rdd.map(lambda x: (int(x[0]), int(x[1]), x[2], x[3], int(x[4])))

In [ ]:
df = spark.createDataFrame(rdd, schema=schema)

In [ ]:
df.show(5)

## Поработаем с искусственным access логом

In [ ]:
!hdfs dfs -tail /lectures/lecture02/data/logsM.txt

In [ ]:
from pyspark.sql.types import LongType

log_schema = StructType(fields=[
    StructField("ip", StringType()),
    StructField("timestamp", LongType()),
    StructField("url", StringType()),
    StructField("size", IntegerType()),
    StructField("code", IntegerType()),
    StructField("ua", StringType())
])

In [ ]:
log = spark.read.csv("/lectures/lecture02/data/logsM.txt", sep="\t", schema=log_schema).cache()

In [ ]:
log

In [ ]:
log.rdd.getNumPartitions()

In [ ]:
log = log.repartition(4)

In [ ]:
log.show(5, vertical=True, truncate=False)

## Projections и filters
Projection - это подмножемтво колокнок

Filter - это подмножество строк

In [ ]:
log.schema.fieldNames()

In [ ]:
log.select(["ip", "timestamp", "url"])

In [ ]:
log.select(*log.schema.fieldNames()[:3]).show(5)

In [ ]:
log.select("ip", "code").show(5)

In [ ]:
log.select(log.ip, log.code).show(5)

In [ ]:
log.ip

## Aliasing

In [ ]:
log.select(log.ip,
           log.code.alias("response")).show(5)

In [ ]:
import pyspark.sql.functions as f

In [ ]:
log.select("ip", 
           f.col("code").alias("response")).show(5)

## Good ol' Pandas

In [ ]:
log[["ip", "code"]].show(5)

In [ ]:
log[[log.ip, log.code.alias("response")]].show(5)

## Filtering

In [ ]:
log.where("code = 200").show(5)

In [ ]:
log.filter(log.code == 200).show(5, truncate=False)

In [ ]:
log.filter("code == 200 AND url LIKE '%rambler%'").show(5, truncate=False, vertical=True)

In [ ]:
log.filter((log.code.isin([200, 404])) & (log.url.like("%rambler%"))).show(5)

## Good ol' Pandas

In [ ]:
log[(log.code == 200) & (log.url.like("%rambler%"))].show(5)

## Все вместе

In [ ]:
log[(log.code == 200) & (log.url.like("%rambler%"))][["ip", "code"]].show(5)

## Что же насчет SQL?! Он тут как тут)

In [ ]:
query = """
SELECT ip, code FROM log_table 
WHERE code == 200 AND url LIKE '%rambler%'
"""

In [ ]:
spark.sql(query).show(5)

### Надо зарегистрировать свой DataFrame как table

In [ ]:
log.registerTempTable("log_table")

In [ ]:
spark.sql(query).show(5)

## Как Spark будет выполнять запрос?

In [ ]:
spark.sql(query).explain(5)

## Функции есть 3 типа

+ mapping (one to one)
+ generating (one to many)
+ aggregating (many to one)

In [ ]:
log.select("ua", f.length("ua")).show(5)

In [ ]:
log.select("ua", f.length("ua").alias("length")).show(5)

In [ ]:
log.select(f.concat("url", "?utm_medium=email")).show(5)

**`concat` необходим `Column` как аргумент. `lit` создает новую `Column` из literal**

In [ ]:
log.select(f.concat("url", f.lit("?utm_medium=email")).alias("newurl")).show(5, False)

## Explosions!

In [ ]:
log.select("ua", f.split("ua", " ").alias("word_list")).show(5, False, True)

**Можно выбирать отдельные элементы из list!**

In [ ]:
log.select("ua", f.split("ua", " ").alias("word_list"))\
   .select(f.col("word_list")[0], f.col("word_list")[1])\
   .show(5)

In [ ]:
log.select("ua", f.split("ua", " ").alias("word_list"))\
   .select(f.explode("word_list").alias("word"))\
   .groupby("word").count()\
   .orderBy("count", ascending=False)\
   .show(5)

## Joins

In [ ]:
!hdfs dfs -tail /lectures/lecture02/data/ipDataM.txt

In [ ]:
ip_schema = StructType(fields=[
    StructField("ip", StringType()),
    StructField("region", StringType())
])

In [ ]:
ips = spark.read.csv("/lectures/lecture02/data/ipDataM.txt", schema=ip_schema, sep="\t").cache()

In [ ]:
ips.show(5)

## Есть трюк, который отключает автоматические broadcast'ы для joins. 

In [ ]:
spark.sql("SET spark.sql.autoBroadcastJoinThreshold = 100500")

In [ ]:
log_with_regions = log.join(ips, on="ip", how="inner")

In [ ]:
log_with_regions

In [ ]:
log_with_regions.show(5)

In [ ]:
log_with_regions.rdd.getNumPartitions()

In [ ]:
spark.conf.get("spark.sql.shuffle.partitions")

In [ ]:
log_with_regions = log_with_regions.coalesce(4).cache()

## Query planner использует SortMergeJoin по дефолту

In [ ]:
log_with_regions.explain(extended=True)

## Broadcast hint

In [ ]:
log_with_regions = log.join(f.broadcast(ips), on="ip", how="inner")

In [ ]:
log_with_regions.explain()

## Или можно использовать метод `hint()`

In [ ]:
log_with_regions = log.join(ips.hint("broadcast"), on="ip", how="inner").cache()

In [ ]:
log_with_regions.explain(True)

## Aggregations

## Aggregation pipeline выглядит следующим образом:
```python
df.groupBy(*cols)\
  .agg(*expressions)
```

In [ ]:
log_with_regions.groupBy("region")\
                .agg(f.count("ip").alias("count"))\
                .orderBy("count", ascending=False)\
                .show(10)

In [ ]:
log_with_regions.groupBy("region")\
                .count()\
                .withColumnRenamed("count", "row_count")\
                .orderBy("row_count", ascending=False)\
                .show(10)

In [ ]:
length_stat = log_with_regions.groupBy(f.length("url").alias("url_length"))\
                              .agg(f.count("*").alias("row_count"))\
                              .orderBy("row_count", ascending=False)\
                              .toPandas()

In [ ]:
length_stat

In [ ]:
length_stat.plot(kind="bar", x="url_length", y="row_count")

In [ ]:
log_with_domains = log_with_regions.withColumn("domain", f.regexp_extract("url", "http:\/\/(.*)\/", 1))

In [ ]:
log_with_domains.show(5, False, True)

## Есть ли корреляция между `url_length` и `domain`?

In [ ]:
length_stat = log_with_domains.groupBy(f.length("url").alias("url_length"), "domain")\
                              .agg(f.count("*").alias("row_count"))\
                              .orderBy("row_count", ascending=False)\
                              .toPandas()

In [ ]:
length_stat

In [ ]:
log_with_domains[log_with_domains.domain == "lenta.ru"][["url"]].show(5, False)

In [ ]:
log_with_domains.withColumn("url_length", f.length("url")).corr("domain", "url_length")

## User Defined Functions
UDF может иметь один из следующих типов:
+ **SCALAR**. scalar UDF задает transformation: Одну или несколько `pandas.Series` -> `pandas.Series`. Scalar UDFs используются с `pyspark.sql.DataFrame.withColumn()` и `pyspark.sql.DataFrame.select()`
+ **GROUPED_MAP**.grouped map UDF задает transformation: A `pandas.DataFrame` -> A `pandas.DataFrame`. Grouped map UDFs используются с `pyspark.sql.GroupedData.apply()`

In [ ]:
log_with_domains[["domain"]].distinct().collect()

In [ ]:
@f.pandas_udf(IntegerType())
def encode_domain(domains):
    mapping = {
        'lenta.ru': 0,
        'newsru.com': 1,
        'news.mail.ru': 2,
        'news.yandex.ru': 3,
        'news.rambler.ru': 4
    }
    return domains.apply(lambda x: mapping.get(x))

In [ ]:
log_with_domains.withColumn("domain_encoded", encode_domain("domain")).show(5, vertical=True, truncate=False)

In [ ]:
log_with_domains.withColumn("domain_encoded", encode_domain("domain"))\
                .withColumn("url_length", f.length("url"))\
                .corr("domain_encoded", "url_length")

In [ ]:
log_with_domains.filter(f.isnull("url")).count()

In [ ]:
log_with_domains.groupby("domain").agg(f.min(f.length("url")), f.max(f.length("url"))).show()

In [ ]:
spark.stop()